In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
!pip install --upgrade spacy
!pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████                   | 834.1 MB 1.3 MB/s eta 0:15:53tcmalloc: large alloc 1147494400 bytes == 0x39818000 @  0x7f6105f39615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51740e 0x58f2a7 0x517947 0x5b41c5 0x58f49e
     |████████████████▌               | 1055.7 MB 1.3 MB/s eta 0:12:30tcmalloc: large alloc 1434370048 bytes == 0x7de6e000 @  0x7f6105f39615 0x58e046 0x4f2e5e 0x4d19df 0x51b31c 0x5b41c5 0x58f49e 0x51b221 0x5b41c5 0x58f49e 0x51837f 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4cfabb 0x517aa0 0x4ba70a 0x538136 0x590055 0x51b180 0x

In [ ]:
!python -m spacy convert /gdrive/MyDrive/NER_UK/fixed-split/test.iob /tmp
!python -m spacy convert /gdrive/MyDrive/NER_UK/fixed-split/train.iob /tmp

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (4045 documents): /tmp/test.spacy
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (8762 documents): /tmp/train.spacy


In [ ]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText && pip install .

Cloning into 'fastText'...
remote: Enumerating objects: 3930, done.
remote: Counting objects: 100% (943/943), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 3930 (delta 854), reused 806 (delta 806), pack-reused 2987
Receiving objects: 100% (3930/3930), 8.24 MiB | 23.64 MiB/s, done.
Resolving deltas: 100% (2505/2505), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/fastText
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux

In [ ]:
from fasttext import load_model
import argparse
import errno

def convert_bin_to_vec(model, output):
    f = load_model(model)
    words = f.get_words()
    with open(output, "w") as fp_out:
        fp_out.write(str(len(words)) + " " + str(f.get_dimension()) + "\n")
        for w in words:
            v = f.get_word_vector(w)
            vstr = ""
            for vi in v:
                vstr += " " + str(vi)
            try:
                fp_out.write(w + vstr + "\n")
            except IOError as e:
                if e.errno == errno.EPIPE:
                    pass

In [ ]:
import os.path
!mkdir /content/SpaCyResults/
f = "/gdrive/MyDrive/UberGrid/vectors/cbow_weighted/baseline_skipgram.bin"
bf, _ = os.path.splitext(os.path.basename(f))

if not os.path.exists(f"/gdrive/MyDrive/SpaCyResults/{bf}"):
    !rm -rf /content/FastText
    !mkdir /content/FastText
    txt_vectors = os.path.join("/content/FastText/", bf + ".vec")

    convert_bin_to_vec(f, txt_vectors)
    !python -m spacy init vectors uk {txt_vectors} /content/FastText

    !python -m spacy train /gdrive/MyDrive/SpaCyResults/ner_uk_cpu_init.config --output /content/SpaCyResults/{bf} --gpu-id 0
    !rm -rf /content/SpaCyResults/{bf}/model-last
    !mv /content/SpaCyResults/{bf} /gdrive/MyDrive/SpaCyResults/

!rm -rf /content/SpaCyResults/

In [ ]:
import os.path
from glob import glob

!mkdir /content/SpaCyResults/

for f in glob("/gdrive/MyDrive/UberGrid/UberText_punct/*.bin"):
    bf, _ = os.path.splitext(os.path.basename(f))

    if os.path.exists(f"/gdrive/MyDrive/UberGrid/UberText_punct/SpacyEval/{bf}"):
        print(f"Skipping {bf} as their already exists")
        continue

    print(f"Converting vectors from {bf}")

    !rm -rf /content/FastText
    !mkdir /content/FastText
    txt_vectors = os.path.join("/content/FastText/", bf + ".vec")

    convert_bin_to_vec(f, txt_vectors)
    !python -m spacy init vectors uk {txt_vectors} /content/FastText

    !python -m spacy train /gdrive/MyDrive/SpaCyResults/ner_uk_cpu_init.config --output /content/SpaCyResults/{bf} --gpu-id 0
    !rm -rf /content/SpaCyResults/{bf}/model-last
    !mv /content/SpaCyResults/{bf} /gdrive/MyDrive/UberGrid/UberText_punct/SpacyEval/

Converting vectors from ubertext2.0.skipgrams.champion
ℹ Creating blank nlp object for language 'uk'
[2022-09-27 13:12:29,039] [INFO] Reading vectors from /content/FastText/ubertext2.0.skipgrams.champion.vec
INFO:spacy:Reading vectors from /content/FastText/ubertext2.0.skipgrams.champion.vec
4398076it [00:36, 120612.62it/s]
[2022-09-27 13:13:05,801] [INFO] Loaded vectors from /content/FastText/ubertext2.0.skipgrams.champion.vec
INFO:spacy:Loaded vectors from /content/FastText/ubertext2.0.skipgrams.champion.vec
✔ Successfully converted 4398076 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/content/FastText
✔ Created output directory:
/content/SpaCyResults/ubertext2.0.skipgrams.champion
ℹ Saving to output directory:
/content/SpaCyResults/ubertext2.0.skipgrams.champion
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-09-27 13:15:20,854] 

In [ ]:
import re

from csv import DictWriter
from pathlib import Path
import os.path
from glob import glob
import json

mask = r"algo-([^.]+)\.epochs-(\d+)\.subwords-(\d+)\.\.(\d+)\.wordngram-(\d+).neg_sampling-(\d+)"

weighed = r"subword(\d+)-(\d+)\.cbow_weighted.epoch(\d+)"


results_path = Path("/gdrive/MyDrive/UberGrid/UberText_punct/SpacyEval/")

with (results_path / "extrinsic_eval_punct.csv").open("w") as fp_out:
    w = DictWriter(fp_out, fieldnames=["vectors_file", "vectors_params", "ents_f", "ents_p", "ents_r"])

    w.writeheader()

    for f in glob("/gdrive/MyDrive/UberGrid/UberText_punct/*.bin"):
        bf, _ = os.path.splitext(os.path.basename(f))
        meta_file = results_path / bf / "model-best/meta.json"
        if not os.path.exists(meta_file):
            print(f"Cannot find file {meta_file}, skipping")
            continue

        m = re.search(mask, bf)
        if not m or not (re.search(weighed, bf)):
           print(f"Cannot parse filename {bf}")
           continue

        groups = m.groups()

        with open(meta_file, "r") as fp:
            meta = json.load(fp)

        w.writerow({
            "vectors_file": bf,
            "vectors_params": f"{groups[0]};{groups[1]};{groups[2]}-{groups[3]};{groups[4]};{groups[5]}",
            "ents_f": meta["performance"]["ents_f"],
            "ents_p": meta["performance"]["ents_p"],
            "ents_r": meta["performance"]["ents_r"],
        })